In [275]:
pip install dash

In [276]:
pip install dash_bootstrap_components

In [277]:
pip install pyngrok

In [278]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [279]:
import dash
from dash import dcc, html, Dash, Output, Input
import dash_bootstrap_components as dbc
from pyngrok import ngrok

In [280]:
!pgrep ngrok

45888


In [283]:
!kill 45888

In [284]:
ngrok.set_auth_token('2gIlIcVHuAP0SLHUeijX1RLX5it_59uQsFzeSW93RkPaM2BQs')
public_url=ngrok.connect(addr='8050')
print('url:', public_url)

url: NgrokTunnel: "https://811a-34-23-134-49.ngrok-free.app" -> "http://localhost:8050"


In [285]:
companies = {'google': 'GOOGL.csv', 'tesla': 'TSLA.csv', 'ibm': 'IBM.csv'}
dataframes = {company: pd.read_csv(filename) for company, filename in companies.items()}

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

page1_layout=dbc.Container(fluid=True, children=[
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(id='multi_stock_selector1',options=[{'label': company.capitalize(), 'value': company} for company in companies],
                              value=['google','tesla'], multi=True),
            dcc.Graph(id='multi_stock_selector_graph1', figure={})], width=6),

        dbc.Col([
            dcc.Dropdown(id='multi_stock_selector2',options=[{'label': company.capitalize(), 'value': company} for company in companies],
                              value=['google','tesla'], multi=True),
            dcc.Graph(id='multi_stock_selector_graph2', figure={})], width=6)

    ]),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(id='multi_stock_selector3',options=[{'label': company.capitalize(), 'value': company} for company in companies],
                              value=['google','tesla'], multi=True),
            dcc.Graph(id='multi_stock_selector_graph3', figure={})], width=12)
    ])

  ])


@app.callback(
    Output('multi_stock_selector_graph1', 'figure'),
    Input('multi_stock_selector1', 'value')
)
def update_graph1(companies):
    fig = go.Figure()

    # Iterate over the selected companies
    for company in companies:
        # Get the dataframe for the current company
        selected_df = dataframes[company]

        # Add a trace for the close prices of the current company
        fig.add_trace(go.Scatter(x=selected_df['Date'], y=selected_df['Close'], mode='lines', name=company.capitalize()))

    # Update layout
    fig.update_layout(
        title='Close Price Comparison',
        xaxis_title='Date',
        yaxis_title='Close Price',
        legend_title='Company'
    )

    return fig



@app.callback(
    Output('multi_stock_selector_graph2', 'figure'),
    Input('multi_stock_selector2', 'value')
)
def update_graph2(companies):
    fig = go.Figure()

    # Iterate over the selected companies
    for company in companies:
        # Get the dataframe for the current company
        selected_df = dataframes[company]

        # Add a trace for the close prices of the current company
        fig.add_trace(go.Scatter(x=selected_df['Date'], y=selected_df['Volume'], mode='lines', name=company.capitalize()))

    # Update layout
    fig.update_layout(
        title='Stock Volume Comparison',
        xaxis_title='Date',
        yaxis_title='Volume',
        legend_title='Company'
    )

    return fig



@app.callback(
    Output('multi_stock_selector_graph3', 'figure'),
    Input('multi_stock_selector3', 'value')
)
def update_graph4(companies):
    fig = go.Figure()

    # Iterate over the selected companies
    for company in companies:
        # Get the dataframe for the current company
        selected_df = dataframes[company]

        # Add a trace for the close prices of the current company
        fig.add_trace(go.Scatter(x=selected_df['Date'], y=selected_df['Close']*selected_df['Volume'], mode='lines', name=company.capitalize()))

    # Update layout
    fig.update_layout(
        title='Stock Total Traded',
        xaxis_title='Date',
        yaxis_title='Total Traded',
        legend_title='Company'
    )

    return fig


In [286]:
# Page 2 layout
page2_layout=dbc.Container(fluid=True, children=[

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(id='stock_selector',options=[{'label': company.capitalize(), 'value': company} for company in companies],
                              value='google', multi=False),
            dcc.Graph(id='stock_selector_graph', figure={})], width=12)
    ]),
  dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='stock_selection',
                options=[{'label': company.capitalize(), 'value': company} for company in dataframes.keys()],
                value=list(dataframes.keys())[0:2],  multi=True),
            dcc.Graph(id='correlation_graph', figure={})
        ], width=12)
    ])
])




@app.callback(
    Output('stock_selector_graph', 'figure'),
    Input('stock_selector', 'value')
)
def update_graph3(company):
  selected_df = dataframes[company]
  fig=go.Figure(data=[go.Candlestick(x=selected_df["Date"],
                                   open=selected_df["Open"],
                                   close=selected_df["Close"],
                                   high=selected_df["High"],
                                   low=selected_df["Low"])])
  fig.update_layout(xaxis_rangeslider_visible=False)
  return fig

@app.callback(
    Output('correlation_graph', 'figure'),
    Input('stock_selection', 'value')
)
def update_correlation(selected_stocks):
    if len(selected_stocks) == 1:
        stock = selected_stocks[0]
        if stock in dataframes:
            # If only one stock is selected, display a line plot of its close prices
            fig = go.Figure(data=go.Scatter(x=dataframes[stock].index,
                                            y=dataframes[stock]['Close'],
                                            mode='lines',
                                            marker=dict(color='blue'),
                                            ))
            fig.update_layout(title=f'Correlation: 1.00 (Single Stock: {stock})')
            return fig

    elif len(selected_stocks) == 2:
        # Filter data to include only selected stocks
        filtered_data = {stock: dataframes[stock]['Close'] for stock in selected_stocks if stock in dataframes}

        # Combine the close prices of the selected stocks into a DataFrame
        combined_close_prices = pd.concat(filtered_data, axis=1)

        # Calculate the correlation between the selected stocks' close prices
        correlation = combined_close_prices.corr().iloc[0, 1]

        # Create a scatter plot to visualize the correlation
        fig = go.Figure(data=go.Scatter(x=combined_close_prices[selected_stocks[0]],
                                        y=combined_close_prices[selected_stocks[1]],
                                        mode='markers',
                                        marker=dict(color='blue'),
                                        ))

        # Add correlation information to the plot title
        fig.update_layout(title=f'Correlation between {selected_stocks[0]} and {selected_stocks[1]}: {correlation:.2f}')

        return fig


In [287]:
import plotly.figure_factory as ff

In [288]:
# Third page layout
page3_layout = dbc.Container(fluid=True, children=[
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='stock_selector_returns',
                options=[{'label': company.capitalize(), 'value': company} for company in companies],
                value=['google', 'ibm'],
                multi=True
            ),
            dcc.Graph(id='returns_graph', figure={})
        ], width=6),
        dbc.Col([
            dcc.Dropdown(
                id='stock_selector_volatility',
                options=[{'label': company.capitalize(), 'value': company} for company in companies],
                value=['google', 'ibm'],
                multi=True
            ),
            dcc.Graph(id='volatility_graph', figure={})
        ], width=6)
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='stock_selector_density',
                options=[{'label': company.capitalize(), 'value': company} for company in companies],
                value=['google', 'ibm'],
                multi=True
            ),
            dcc.Graph(id='density_graph', figure={})
        ], width=12)
    ])
])
# Callback to update returns plot, volatility histogram, and density estimation
@app.callback(
    [Output('returns_graph', 'figure'),
     Output('volatility_graph', 'figure'),
     Output('density_graph', 'figure')],
    [Input('stock_selector_returns', 'value'),
     Input('stock_selector_volatility', 'value'),
     Input('stock_selector_density', 'value')]
)
def update_returns_analysis(selected_stocks_returns, selected_stocks_volatility, selected_stocks_density):
    return (
        create_returns_plot(selected_stocks_returns),
        create_volatility_plot(selected_stocks_volatility),
        create_density_plot(selected_stocks_density)
    )

def create_returns_plot(companies):
    fig = go.Figure()
    for company in companies:
        df = dataframes[company]
        df['Return'] = df['Close'].pct_change() * 100  # Calculate daily percentage change
        fig.add_trace(go.Scatter(x=df['Date'], y=df['Return'], mode='lines', name=company.capitalize() + ' Return'))
    fig.update_layout(title='Daily Percentage Change (Returns)')
    return fig

def create_volatility_plot(companies):
    fig = go.Figure()
    for company in companies:
        df = dataframes[company]
        df['Return'] = df['Close'].pct_change() * 100
        fig.add_trace(go.Histogram(x=df['Return'], name=company.capitalize() + ' Volatility'))
    fig.update_layout(title='Volatility (Histogram of Returns)')
    return fig

def create_density_plot(companies):
    fig = go.Figure()

    # Define a color scale for KDE plots
    colors = ['blue', 'green', 'red', 'purple', 'orange', 'yellow']  # Add more colors if needed

    for i, company in enumerate(companies):
        df = dataframes[company]
        df['Return'] = df['Close'].pct_change() * 100  # Calculate daily percentage change
        df['Return'] = df['Return'].replace([np.inf, -np.inf], np.nan)  # Replace inf with NaN
        df['Return'].dropna(inplace=True)  # Remove NaN values
        df['Return'] = df['Return'].replace([np.nan], 0)  # Replace remaining NaN values with 0

        hist_fig = ff.create_distplot(
            [df['Return']],
            histnorm='probability',
            group_labels=[company.capitalize()],
            show_rug=False,
            show_hist=False,
            colors=[colors[i]]  # Set color for this KDE plot
        )

        for trace in hist_fig['data']:
            fig.add_trace(trace)

    fig.update_layout(title='Volatility (KDE of Returns)')
    return fig

In [289]:
page4_layout = dbc.Container(fluid=True, children=[
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='company_selector_5',
                options=[{'label': company.capitalize(), 'value': company} for company in companies],
                value='google',
                multi=False
            ),
            dcc.Graph(id='scatter_plot', figure={})
        ], width=12),
    ]),
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H5("Explanation:"),
                html.P("This scatter plot visualizes the relationship between trading volume and closing price of the selected stock. The size of each point represents the absolute value of daily returns, while the color indicates the direction and magnitude of price changes (negative values in red, positive values in green)."),
            ])
        ], width=12)
    ])
])

# Callback to update scatter plot
@app.callback(
    Output('scatter_plot', 'figure'),
    [Input('company_selector_5', 'value')]
)
def update_scatter_plot(selected_company):
    df = dataframes[selected_company]
    df['Returns'] = df['Close'].diff()
    df["Abs_Returns"] = df['Returns'].abs()
    df = df.dropna()

    fig = px.scatter(df,
                     x="Volume",
                     y="Close",
                     size="Abs_Returns",
                     color="Returns",
                     color_continuous_scale=['red', 'lightgray', 'green'],
                     hover_data=["Date", "Returns", "Abs_Returns"],
                     title=f'Scatter Plot of {selected_company.capitalize()} Volume, Close, Returns and Price Change')
    fig.update_layout(xaxis_title="Volume", yaxis_title="Close")

    return fig

In [290]:
app.layout = dbc.Container(fluid=True, children=[
    dbc.Row([
        dbc.Col([html.H1('Multipage dashboard. Stock Market Analysis: Tesla, Google and IBM', className="text-center")], width=12)
    ]),
    dbc.Row(
        [
            dbc.Col(
                [
                    dbc.Nav(
                        [dbc.NavLink(page['name'], href=page['path'], active='exact') for page in [
                            {'name': 'page1', 'path': '/'},
                            {'name': 'page2', 'path': '/cc'},
                            {'name': 'page3', 'path': '/returns'},
                            {'name': 'page4', 'path': '/scatter'}
                        ]],
                        vertical=True,
                        pills=True
                    )
                ],
                width=2
            ),
            dbc.Col([dcc.Location(id='url', refresh=True), html.Div(id='page-content')], width=10)
        ]
    )
])

# Define callback to switch between pages

@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/':
        return page1_layout
    elif pathname == '/cc':
        return page2_layout
    elif pathname == '/returns':
        return page3_layout
    elif pathname == '/scatter':
        return page4_layout
    else:
        return '404 Page Not Found'






---


---



---



In [292]:
!pip freeze > requirements.txt

In [293]:
%%writefile Procfile
web:gunicorn app:server

# Run the app

if __name__ == '__main__':
    app.run_server(port=8050, debug=True)

Writing Procfile
